# Model Runner

This workbook is a test bed for running models.

# Imports & Setup

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [93]:
from fastai import *
from fastai.vision import *
import json
import aiohttp
from PIL import Image as PILImage

from io import BytesIO
from IPython.display import Image, display

In [5]:
bs = 64
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

In [6]:
modelDir = os.path.abspath(os.path.join(os.getcwd(), '../models'))
modelName = 'pet_classifier'

In [125]:
class ModelRunner:
    def __init__(self, json_file_path):
        self.json_file_path = json_file_path
        self.model_dir = os.path.dirname(json_file_path)
        self.model_name = os.path.splitext(os.path.basename(self.json_file_path))[0]
        self.loaded = False
                
    def load(self):        
        with open(self.json_file_path) as f:
            json_data = json.load(f)
            self.learner = self.create_learner_from_json(json_data['model'])       
            self.learner.model_dir = self.model_dir
            self.learner.load(self.model_name)    
            self.loaded = True
                
    def create_learner_from_json(self, model_json):        
        if model_json["model_type"] == 'image_classificiation':            
            data = ImageDataBunch.single_from_classes("/tmp", model_json["image_classes"], tfms=get_transforms(), size=224).normalize(imagenet_stats)                       
            if model_json["image_architecture"] == 'resnet34': return create_cnn(data, models.resnet34)
            elif model_json["image_architecture"] == 'resnet50': return create_cnn(data, models.resnet50)                     
            else: raise Exception('invalid image_architecture') 
        else:
            raise Exception('invalid model_type') 
            
        
    def predict_image(self, img):
         if self.loaded == False: raise Exception("must call load() before predicting")
        
         pred_class, pred_idx, outputs = self.learner.predict(img)
         formatted_outputs = ["{:.1f}%".format(value) for value in [x * 100 for x in torch.nn.functional.softmax(outputs, dim=0)]]
         pred_probs = sorted(
                zip(self.learner.data.classes, map(str, formatted_outputs)),
                key=lambda p: p[1],
                reverse=True)
         return pred_class, pred_probs, img
    

In [129]:
async def get_image(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
           bytes = await response.read()
           return open_image(BytesIO(bytes))

#url = "https://news.nationalgeographic.com/content/dam/news/2018/05/17/you-can-train-your-cat/02-cat-training-NationalGeographic_1484324.ngsversion.1526587209178.adapt.1900.1.jpg"
#url = "https://imagesvc.timeincapp.com/v3/mm/image?url=https%3A%2F%2Fpeopledotcom.files.wordpress.com%2F2018%2F02%2Ftwo-tone-cat.jpg%3Fw%3D2000&w=700&c=sc&poi=face&q=85"
#url = "https://s3.amazonaws.com/cdn-origin-etr.akc.org/wp-content/uploads/2017/11/12222621/Yorkshire-Terrier-On-White-10.jpg"
url = "https://i1.wp.com/www.k9web.com/wp-content/uploads/2018/03/dog-2785087_1280.jpg?ssl=1"
url_img = await get_image(url)
#display(url_img)

json_file_path = os.path.join(modelDir, "pet_classifier_stage_1.json")
mr = ModelRunner(json_file_path); 
mr.load()
pred_classes, pred_probs, img = mr.predict_image(url_img); pred_classes, pred_probs

(Category yorkshire_terrier,
 [('yorkshire_terrier', '7.0%'),
  ('Abyssinian', '2.6%'),
  ('Bengal', '2.6%'),
  ('Birman', '2.6%'),
  ('Bombay', '2.6%'),
  ('British_Shorthair', '2.6%'),
  ('Egyptian_Mau', '2.6%'),
  ('Maine_Coon', '2.6%'),
  ('Persian', '2.6%'),
  ('Ragdoll', '2.6%'),
  ('Russian_Blue', '2.6%'),
  ('Siamese', '2.6%'),
  ('Sphynx', '2.6%'),
  ('american_bulldog', '2.6%'),
  ('american_pit_bull_terrier', '2.6%'),
  ('basset_hound', '2.6%'),
  ('beagle', '2.6%'),
  ('boxer', '2.6%'),
  ('chihuahua', '2.6%'),
  ('english_cocker_spaniel', '2.6%'),
  ('english_setter', '2.6%'),
  ('german_shorthaired', '2.6%'),
  ('great_pyrenees', '2.6%'),
  ('havanese', '2.6%'),
  ('japanese_chin', '2.6%'),
  ('keeshond', '2.6%'),
  ('leonberger', '2.6%'),
  ('miniature_pinscher', '2.6%'),
  ('newfoundland', '2.6%'),
  ('pomeranian', '2.6%'),
  ('pug', '2.6%'),
  ('saint_bernard', '2.6%'),
  ('samoyed', '2.6%'),
  ('scottish_terrier', '2.6%'),
  ('shiba_inu', '2.6%'),
  ('staffordshire_bu